In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "data"
image_files = os.listdir(dataset_path)  # 获取文件夹中的所有文件名
image_files = [file for file in image_files if file.endswith(".jpg")]  # 仅选择以.jpg结尾的文件

# 定义高斯滤波器参数
kernel_size = (5, 5)  # 滤波器大小
sigma = 3.0  # 标准差

# 定义二值化参数
threshold = 100  # 阈值

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path)

    # 进行高斯滤波
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 二值化图像
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # 使用形态学操作去除大面积黑色区域
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # 反转图像的黑白色调
    inverted = cv2.bitwise_not(opening)

    # 连通组件标记
    _, labels, stats, _ = cv2.connectedComponentsWithStats(inverted, connectivity=8)

    # 根据连通组件的像素数量进行区域过滤
    filtered_mask = np.zeros_like(inverted)
    for label in range(1, np.max(labels)+1):
        if stats[label, cv2.CC_STAT_AREA] < 2500:
            filtered_mask[labels == label] = 255

    # 保存掩码图像
    mask_filename = "mask_" + img_file
    mask_path = "mask/" + mask_filename
    cv2.imwrite(mask_path, filtered_mask)

    # 添加到掩码列表
    masks.append(inverted)

    print("Mask image", mask_filename, "saved successfully.")

# 显示结果
#for i in range(len(images)):
#    cv2.imshow("Original Image", images[i])
#    cv2.imshow("Mask", masks[i])
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()

Mask image mask_P041_usp_data_interval_4_3_8612C7D2-DACE-43AD-AED4-7C87FCDBCEDE.jpeg saved successfully.


In [ ]:
import zipfile

zip_file_name = "UNet-image001.zip" # image 50张图像; image1 600张图像; image2 650张图像; test 13张图像

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall()

# 阈值分割

In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)  # 获取文件夹中的所有文件名
image_files = [file for file in image_files if file.endswith(".jpg")]  # 仅选择以.jpg结尾的文件

# 定义高斯滤波器参数
kernel_size = (5, 5)  # 滤波器大小
sigma = 3.0  # 标准差

# 定义二值化参数
threshold = 180  # 阈值

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path)

    # 进行高斯滤波
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 二值化图像
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # 使用形态学操作去除大面积黑色区域
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)


    # # 连通组件标记
    # _, labels, stats, _ = cv2.connectedComponentsWithStats(opening, connectivity=8)

    # # 根据连通组件的像素数量进行区域过滤
    # filtered_mask = np.zeros_like(opening)
    # for label in range(1, np.max(labels)+1):
    #     if stats[label, cv2.CC_STAT_AREA] < 7500:
    #         filtered_mask[labels == label] = 255

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "mask1/" + mask_filename
    cv2.imwrite(mask_path, opening)

    # # 添加到掩码列表
    # masks.append(inverted)

    print("Mask image", mask_filename, "saved successfully.")

# 显示结果
#for i in range(len(images)):
#    cv2.imshow("Original Image", images[i])
#    cv2.imshow("Mask", masks[i])
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()

# 边缘检测

In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 定义高斯滤波器参数
kernel_size = (5, 5)
sigma = 3.0

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path)

    # 进行高斯滤波
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 使用Canny边缘检测
    edges = cv2.Canny(gray, threshold1=10, threshold2=160)

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "mask2/" + mask_filename
    cv2.imwrite(mask_path, edges)

    print("Mask image", mask_filename, "saved successfully.")

# 特征提取

局部二值模式（Local Binary Pattern，LBP）或灰度共生矩阵（Gray Level Co-occurrence Matrix，GLCM）

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 定义LBP特征提取的半径和邻域点数
radius = 4
neighbors = 10

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # 使用LBP特征提取
    lbp_image = local_binary_pattern(img, neighbors, radius, method='uniform')

    # 将LBP图像转换为二值图像
    binary_lbp = (lbp_image > 0).astype(np.uint8) * 255

    # 使用形态学操作去除大面积黑色区域
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    opening = cv2.morphologyEx(binary_lbp, cv2.MORPH_OPEN, kernel)

    # 颠倒黑白像素
    inverted_mask = 255 - opening

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "mask3/" + mask_filename
    cv2.imwrite(mask_path, inverted_mask)

    print("Mask image", mask_filename, "saved successfully.")

# K均值聚类（K-means Clustering）

In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path)

    # 将图像转换为一维数组
    img_flat = img.reshape((-1, 3))

    # 使用K均值聚类分割图像
    k = 2  # 聚类数
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(img_flat.astype(np.float32), k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # 将标签图像转换为掩码图像
    mask = labels.reshape(img.shape[:2])
    mask = np.uint8(mask)

    # 将掩码图像二值化
    _, binary_mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 使用形态学操作去除噪声
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "mask4/" + mask_filename
    cv2.imwrite(mask_path, opening)

    print("Mask image", mask_filename, "saved successfully.")

# 轮廓检测方法（如cv2.findContours()函数）

In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 生成掩码图像
masks = []
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # 使用Canny边缘检测
    edges = cv2.Canny(img, threshold1=20, threshold2=160)

    # 找到图像中的轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 创建一个全黑的掩码图像
    mask = np.zeros_like(img)

    # 将找到的轮廓绘制到掩码图像上
    cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

    # # 使用形态学操作去除噪声
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    # opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "mask5/" + mask_filename
    cv2.imwrite(mask_path, mask)

    print("Mask image", mask_filename, "saved successfully.")

# 最终使用的传统方法，结合到一起的

In [ ]:
import os
import cv2
import numpy as np

# 加载图像数据集
dataset_path = "test/"
image_files = os.listdir(dataset_path)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 定义高斯滤波器参数
kernel_size = (1, 1)
sigma = 6.0

# 定义连通域保留的数量
num_to_keep = 7

# 生成掩码图像
for img_file in image_files:
    img_path = dataset_path + "/" + img_file
    img = cv2.imread(img_path)

    # 进行高斯滤波
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 使用Canny边缘检测
    edges = cv2.Canny(gray, threshold1=20, threshold2=160)

    # # 进行直方图均衡化
    # equalized = cv2.equalizeHist(edges)

    # 阈值分割
    _, threshold_mask = cv2.threshold(edges, 190, 245, cv2.THRESH_BINARY)

    # 进行直方图均衡化
    equalized = cv2.equalizeHist(threshold_mask)

    # 使用特征提取
    features = cv2.cornerHarris(equalized, 2, 3, 0.004)
    features = cv2.dilate(features, None)
    feature_mask = np.zeros_like(equalized)
    feature_mask[features > 0.00001 * features.max()] = 255

    # 连通组件标记
    _, labels, stats, _ = cv2.connectedComponentsWithStats(feature_mask, connectivity=8)

    # 根据连通组件的面积进行排序
    sorted_indices = np.argsort(stats[:, cv2.CC_STAT_AREA])[::-1]
    print(sorted_indices)

    # 创建一个全黑的掩码图像
    mask = np.zeros_like(equalized)

    # 保留指定数量的连通域
    for i in range(1, num_to_keep + 1):
        if i < len(sorted_indices):
            label = sorted_indices[i]
            mask[labels == label] = 255

    # 计算mask的补集
    mask_complement = cv2.bitwise_not(mask)

    # 将补集与feature_mask进行按位与操作
    subtracted_mask = cv2.bitwise_and(feature_mask, mask_complement)

    # # 将不同方法得到的掩码图像合并
    # combined_mask = cv2.bitwise_or(feature_mask, mask)

    # 保存掩码图像
    mask_filename = img_file
    mask_path = "combined_masks1/" + mask_filename
    cv2.imwrite(mask_path, subtracted_mask)

    print("Mask image", mask_filename, "saved successfully.")

# 评价指标

In [ ]:
import os
import numpy as np
import cv2
from sklearn.metrics import average_precision_score

binarized_masks = []
# for mask in masks:
#     if mask is not None:  # 检查 mask 是否为 None
#         binarized_mask = mask > threshold
#         binarized_masks.append(binarized_mask)
#     else:
#         # 如果 mask 为 None，可以添加一个全 False 的占位符
#         binarized_masks.append(np.zeros_like(mask, dtype=bool))

def binarize_masks(masks, threshold=0.5):
    binarized_masks = [mask > threshold for mask in masks]
    return binarized_masks

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    iou = intersection / union if union > 0 else 0.0
    return iou

# Load predicted masks from mask0 folder
predicted_mask_folder = 'mask1/'  # orig_new1 combined_masks1
predicted_mask_files = os.listdir(predicted_mask_folder)
predicted_masks = [cv2.imread(os.path.join(predicted_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in predicted_mask_files]
predicted_masks = binarize_masks(predicted_masks)
print(predicted_masks)  # 打印读取的图像，检查是否成功读取

# Load true masks from mask1 folder
true_mask_folder = 'test_mask/'
true_mask_files = os.listdir(true_mask_folder)
true_masks = [cv2.imread(os.path.join(true_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in true_mask_files]
true_masks = binarize_masks(true_masks)
# print(true_masks)  # 打印读取的图像，检查是否成功读取

# Initialize metrics
num_classes = 2  # Assuming binary masks
iou_scores = []
precisions = []
recalls = []
f1_scores = []

# Calculate IoU for each class and overall mIoU
for class_idx in range(num_classes):
    ious = []
    class_true_positive = 0
    class_false_positive = 0
    class_false_negative = 0

    for pred_mask, true_mask in zip(predicted_masks, true_masks):
        iou = calculate_iou(pred_mask, true_mask)
        ious.append(iou)

        true_positive = np.logical_and(pred_mask, true_mask).sum()
        false_positive = np.logical_and(pred_mask, ~true_mask).sum()
        false_negative = np.logical_and(~pred_mask, true_mask).sum()

        class_true_positive += true_positive
        class_false_positive += false_positive
        class_false_negative += false_negative

    class_precision = class_true_positive / (class_true_positive + class_false_positive)
    class_recall = class_true_positive / (class_true_positive + class_false_negative)
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall)

    iou_scores.append(sum(ious) / len(ious))
    precisions.append(class_precision)
    recalls.append(class_recall)
    f1_scores.append(class_f1)

# Calculate mean metrics
mean_iou = sum(iou_scores) / num_classes
mean_precision = sum(precisions) / num_classes
mean_recall = sum(recalls) / num_classes
mean_f1 = sum(f1_scores) / num_classes

# Calculate mAP (Average Precision)
true_masks_stacked = np.stack(true_masks, axis=-1)
predicted_masks_stacked = np.stack(predicted_masks, axis=-1)
average_precisions = []
for class_idx in range(num_classes):
    true_class = true_masks_stacked[:, :, class_idx].flatten()
    predicted_class = predicted_masks_stacked[:, :, class_idx].flatten()
    average_precision = average_precision_score(true_class, predicted_class)
    average_precisions.append(average_precision)
mean_average_precision = sum(average_precisions) / num_classes

# Print evaluation metrics
print(f"Mean Intersection over Union (mIoU): {mean_iou}")
print(f"Mean Average Precision (mAP): {mean_average_precision}")
print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")
print(f"Mean F1 Score: {mean_f1}")

# 转换成黑白像素值的mask

In [ ]:
import os
import cv2
import numpy as np

# 设置文件夹路径
orig_folder = "orig/"
orig_new_folder = "orig_new/"

# 确保目标文件夹存在
os.makedirs(orig_new_folder, exist_ok=True)

# 获取原始文件夹中的图像文件列表
image_files = os.listdir(orig_folder)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 遍历图像文件并进行处理
for img_file in image_files:
    img_path = os.path.join(orig_folder, img_file)
    img = cv2.imread(img_path)

    # 将像素值接近黑色的部分变为白色，其余部分变为黑色
    mask_dark = np.all(img <= [125, 125, 125], axis=-1)
    img[mask_dark] = [255, 255, 255]
    img[~mask_dark] = [0, 0, 0]

    # 调整图像尺寸为1280x720
    img_resized = cv2.resize(img, (1280, 720))

    # 保存处理后的图像到orig_new文件夹
    new_img_path = os.path.join(orig_new_folder, img_file)
    cv2.imwrite(new_img_path, img_resized)

    print("Processed and resized image", img_file, "saved successfully.")

Processed and resized image P158.jpg saved successfully.
Processed and resized image P073.jpg saved successfully.
Processed and resized image P012.jpg saved successfully.
Processed and resized image P146.jpg saved successfully.
Processed and resized image P106.jpg saved successfully.
Processed and resized image P152.jpg saved successfully.
Processed and resized image P087.jpg saved successfully.
Processed and resized image P067.jpg saved successfully.
Processed and resized image P080.jpg saved successfully.
Processed and resized image P154.jpg saved successfully.
Processed and resized image Control.jpg saved successfully.
Processed and resized image P135.jpg saved successfully.
Processed and resized image P095.jpg saved successfully.


In [ ]:
import os
import cv2
import numpy as np

# 设置文件夹路径
orig_folder = "predicted_masks2/"
orig_new_folder = "predicted_masks2/"

# 确保目标文件夹存在
os.makedirs(orig_new_folder, exist_ok=True)

# 获取原始文件夹中的图像文件列表
image_files = os.listdir(orig_folder)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 遍历图像文件并进行处理
for img_file in image_files:
    img_path = os.path.join(orig_folder, img_file)
    img = cv2.imread(img_path)

    # 将像素值接近黑色的部分变为白色，其余部分变为黑色
    mask_white = np.all(img >= [0, 0, 10], axis=-1)
    img[mask_white] = [255, 255, 255]
    img[~mask_white] = [0, 0, 0]

    # 调整图像尺寸为1280x720
    img_resized = cv2.resize(img, (1280, 720))

    # 保存处理后的图像到orig_new文件夹
    new_img_path = os.path.join(orig_new_folder, img_file)
    cv2.imwrite(new_img_path, img_resized)

    print("Processed and resized image", img_file, "saved successfully.")

Processed and resized image P154.jpg saved successfully.
Processed and resized image P158.jpg saved successfully.
Processed and resized image P080.jpg saved successfully.
Processed and resized image P012.jpg saved successfully.
Processed and resized image P087.jpg saved successfully.
Processed and resized image P095.jpg saved successfully.
Processed and resized image P152.jpg saved successfully.
Processed and resized image Control.jpg saved successfully.
Processed and resized image P135.jpg saved successfully.
Processed and resized image P073.jpg saved successfully.
Processed and resized image P146.jpg saved successfully.
Processed and resized image P106.jpg saved successfully.
Processed and resized image P067.jpg saved successfully.


In [ ]:
import os
import cv2
import numpy as np

# 设置文件夹路径
orig_folder = "orig"
orig_new_folder = "orig_new"

# 确保目标文件夹存在
os.makedirs(orig_new_folder, exist_ok=True)

# 获取原始文件夹中的图像文件列表
image_files = os.listdir(orig_folder)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 遍历图像文件并进行处理
for img_file in image_files:
    img_path = os.path.join(orig_folder, img_file)
    img = cv2.imread(img_path)

    # 将像素值接近黑色的部分变为白色，其余部分变为黑色
    mask_dark = np.all(img <= [30, 30, 30], axis=-1)
    img[mask_dark] = [255, 255, 255]
    img[~mask_dark] = [0, 0, 0]

    # 保存处理后的图像到orig_new文件夹
    new_img_path = os.path.join(orig_new_folder, img_file)
    cv2.imwrite(new_img_path, img)

    print("Processed image", img_file, "saved successfully.")

# 下载文件

In [ ]:
import shutil

# 指定要下载的文件夹路径
mask_folder_path = "UNet-image001_new"  # 根据你实际的文件夹路径进行修改

# 指定zip文件名
zip_file_name = "UNet-image001_new.zip"

# 将文件夹打包成zip文件
shutil.make_archive("UNet-image001_new", 'zip', mask_folder_path)

'/content/UNet-image001_new.zip'

In [ ]:
from google.colab import files

# 下载zip文件
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 重新写评价指标

In [ ]:
import os
import cv2
import numpy as np

# 设置文件夹路径
orig_folder = "UNet-image/"
orig_new_folder = "UNet-image_new/"

# 确保目标文件夹存在
os.makedirs(orig_new_folder, exist_ok=True)

# 获取原始文件夹中的图像文件列表
image_files = os.listdir(orig_folder)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 遍历图像文件并进行处理
for img_file in image_files:
    img_path = os.path.join(orig_folder, img_file)
    img = cv2.imread(img_path)

    # 将像素值接近黑色的部分变为白色，其余部分变为黑色
    mask_dark = np.all(img <= [125, 125, 125], axis=-1)
    img[mask_dark] = [255, 255, 255]
    img[~mask_dark] = [0, 0, 0]

    # 调整图像尺寸为1280x720
    img_resized = cv2.resize(img, (1280, 720))

    # 保存处理后的图像到orig_new文件夹
    new_img_path = os.path.join(orig_new_folder, img_file)
    cv2.imwrite(new_img_path, img_resized)

    print("Processed and resized image", img_file, "saved successfully.")

Processed and resized image P154.jpg saved successfully.
Processed and resized image P158.jpg saved successfully.
Processed and resized image P080.jpg saved successfully.
Processed and resized image P012.jpg saved successfully.
Processed and resized image P087.jpg saved successfully.
Processed and resized image P095.jpg saved successfully.
Processed and resized image P152.jpg saved successfully.
Processed and resized image Control.jpg saved successfully.
Processed and resized image P135.jpg saved successfully.
Processed and resized image P073.jpg saved successfully.
Processed and resized image P146.jpg saved successfully.
Processed and resized image P106.jpg saved successfully.
Processed and resized image P067.jpg saved successfully.


In [ ]:
import os
import cv2
import numpy as np

# 设置文件夹路径
orig_folder = "UNet-image001/"
orig_new_folder = "UNet-image001_new/"

# 确保目标文件夹存在
os.makedirs(orig_new_folder, exist_ok=True)

# 获取原始文件夹中的图像文件列表
image_files = os.listdir(orig_folder)
image_files = [file for file in image_files if file.endswith(".jpg")]

# 遍历图像文件并进行处理
for img_file in image_files:
    img_path = os.path.join(orig_folder, img_file)
    img = cv2.imread(img_path)

    # 调整图像尺寸为1280x720
    img_resized = cv2.resize(img, (1280, 720))

    # 保存处理后的图像到orig_new文件夹
    new_img_path = os.path.join(orig_new_folder, img_file)
    cv2.imwrite(new_img_path, img_resized)

    print("Processed and resized image", img_file, "saved successfully.")

Processed and resized image P154.jpg saved successfully.
Processed and resized image P158.jpg saved successfully.
Processed and resized image P080.jpg saved successfully.
Processed and resized image P012.jpg saved successfully.
Processed and resized image P087.jpg saved successfully.
Processed and resized image P095.jpg saved successfully.
Processed and resized image P152.jpg saved successfully.
Processed and resized image Control.jpg saved successfully.
Processed and resized image P135.jpg saved successfully.
Processed and resized image P073.jpg saved successfully.
Processed and resized image P146.jpg saved successfully.
Processed and resized image P106.jpg saved successfully.
Processed and resized image P067.jpg saved successfully.


In [ ]:
import os
import numpy as np
import cv2
from sklearn.metrics import average_precision_score

def binarize_masks(masks, threshold=0):
    binarized_masks = [mask > threshold for mask in masks]
    return binarized_masks

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    iou = intersection / union if union > 0 else 0.0
    return iou

# Load predicted masks from mask0 folder
predicted_mask_folder = 'UNet-image001_new/' #mask1 orig_new combined_masks1/ tuned_image_new/
predicted_mask_files = os.listdir(predicted_mask_folder)
predicted_masks = [cv2.imread(os.path.join(predicted_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in predicted_mask_files]
predicted_masks = binarize_masks(predicted_masks)

# Load true masks from mask1 folder
true_mask_folder = 'test_mask/'
true_mask_files = os.listdir(true_mask_folder)
true_masks = [cv2.imread(os.path.join(true_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in true_mask_files]
true_masks = binarize_masks(true_masks)

# Initialize metrics
num_classes = 2  # Assuming binary masks
iou_scores = []
dices = []
precisions = []
recalls = []
f1_scores = []
average_precisions = []

# Calculate metrics
for class_idx in range(num_classes):
    ious = []
    dices = []
    class_true_positive = 0
    class_false_positive = 0
    class_false_negative = 0

    for pred_mask, true_mask in zip(predicted_masks, true_masks):
        iou = calculate_iou(pred_mask, true_mask)
        ious.append(iou)

        dice = 2 * np.logical_and(pred_mask, true_mask).sum() / (pred_mask.sum() + true_mask.sum() + 1e-5)
        dices.append(dice)

        true_positive = np.logical_and(pred_mask, true_mask).sum()
        false_positive = np.logical_and(pred_mask, ~true_mask).sum()
        false_negative = np.logical_and(~pred_mask, true_mask).sum()

        class_true_positive += true_positive
        class_false_positive += false_positive
        class_false_negative += false_negative

    class_precision = class_true_positive / (class_true_positive + class_false_positive)
    class_recall = class_true_positive / (class_true_positive + class_false_negative)
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall)

    iou_scores.append(sum(ious) / len(ious))
    precisions.append(class_precision)
    recalls.append(class_recall)
    f1_scores.append(class_f1)

    # Calculate Average Precision (AP) for each class
    true_masks_stacked = np.stack(true_masks, axis=-1)
    predicted_masks_stacked = np.stack(predicted_masks, axis=-1)
    for class_idx in range(num_classes):
        true_class = true_masks_stacked[:, :, class_idx].flatten()
        predicted_class = predicted_masks_stacked[:, :, class_idx].flatten()
        average_precision = average_precision_score(true_class, predicted_class)
        average_precisions.append(average_precision)

# Calculate mean metrics
mean_iou = sum(iou_scores) / num_classes
mean_dice = np.mean(dices)
mean_precision = sum(precisions) / num_classes
mean_recall = sum(recalls) / num_classes
mean_f1 = sum(f1_scores) / num_classes
mean_average_precision = sum(average_precisions) / num_classes

# Print evaluation metrics
print(f"Mean Intersection over Union (mIoU): {mean_iou}")
print(f"Mean Dice Coefficient: {mean_dice}")
print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")
print(f"Mean F1 Score: {mean_f1}")
print(f"Mean Average Precision (mAP): {mean_average_precision}")

Mean Intersection over Union (mIoU): 0.080487155085578
Mean Dice Coefficient: 0.09934336711083357
Mean Precision: 0.09312831793486062
Mean Recall: 0.877685031231744
Mean F1 Score: 0.16838938340801637
Mean Average Precision (mAP): 0.0429063961564726


In [ ]:
import os
import cv2
import numpy as np
from skimage.measure import label, regionprops

# Create the output folder if it doesn't exist
output_folder = 'UNet-image01_new_new'
os.makedirs(output_folder, exist_ok=True)

# Define the minimum area threshold for removing small connected components
min_area_threshold = 1000

# Iterate through the images in the test folder
for image_filename in os.listdir('UNet-image01_new'):
    if image_filename.endswith('.jpg'):
        # Read the image
        image_path = os.path.join('UNet-image01_new', image_filename)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Threshold the grayscale image to get the binary mask of white areas
        _, binary_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)

        # Label connected components in the binary mask
        labeled_mask, num_labels = label(binary_mask, connectivity=2, return_num=True)

        # Create an array to store the new labeled mask
        new_labeled_mask = np.zeros_like(labeled_mask)

        # Iterate through the labeled components
        for label_idx in range(1, num_labels + 1):
            # Get the region properties for the current component
            props = regionprops(labeled_mask)

            # Check the area of the current component
            if props[label_idx - 1].area >= min_area_threshold:
                # Copy the pixels of the valid component to the new labeled mask
                new_labeled_mask[labeled_mask == label_idx] = label_idx

        # Create the new image with removed small components
        new_image = np.where(new_labeled_mask > 0, 255, 0).astype(np.uint8)

        # Save the new image to the output folder
        new_image_path = os.path.join(output_folder, image_filename)
        cv2.imwrite(new_image_path, new_image)

print("Images with small connected components removed have been saved to 'test_new' folder.")

Images with small connected components removed have been saved to 'test_new' folder.
